In [1]:
import numpy as np
import pandas as pd
from textwrap import wrap
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
import sys, os
from hsbmpy import plot_topic_size, get_max_available_L

In [2]:
directory="/home/jovyan/work/phd/datasets/gtex/10"
os.chdir(directory)
sys.path.append('/home/jovyan/work/phd/')

In [3]:
L = get_max_available_L(directory)

In [4]:
df = pd.read_csv("mainTable.csv", index_col=[0])

# topic size

In [ ]:
for l in range(0,L+1):
    plot_topic_size(directory,l)

## Topic O

In [ ]:
df = pd.read_csv("mainTable.csv", index_col=0,header=0)

In [ ]:
df_mv=pd.DataFrame(data=[df.mean(1), df.var(1),df.apply(lambda x: len([x[x>0]])/float(len(x)), 1)], index=['average', 'var', 'O']).transpose()
df_mv.head()

In [ ]:
for l in range(0,L+1):
    fig = plt.figure(figsize=(15,8))
    ax = fig.subplots(1,2)
    candles = get_candles(directory,l,df_mv,ax[0])
    candlestick2_ohlc(ax[0], candles['open'],candles['high'],candles['low'],candles['close'],width=0.6,colordown='b')
    ax[1].hist((np.array(candles['open'])+np.array(candles['close']))/2, weights=candles['size'], range=(-0.05,1.05), bins=10, histtype='step')
    ax[1].set_xlabel("$O_i", fontsize=18)
    plt.show()
    fig.savefig("%s/topic_Ocandles_level_%d.pdf"%(directory,l))

# Geneontology

In [5]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol
import gseapy as gs

In [ ]:
import importlib, geneontology,tableanalyser
importlib.reload(geneontology)
importlib.reload(tableanalyser)
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [6]:
l=L-1
algorithm = "topsbm"
df_topics = pd.read_csv(f"{directory}/{algorithm}/{algorithm}_level_{l}_topics.csv")

In [7]:
df_symbols= pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_pub_ensembl_id&col=md_ensembl_id&col=md_eg_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", index_col=[0], sep='\t')

In [8]:
def get_sea():
    for g in df_topics.values.ravel()[[str(s)!='nan' for s in df_topics.values.ravel()]]:
        yield get_symbol(g)

In [9]:
with open("gback.txt",'w') as f:
    list(map(lambda x: f.writelines(x+'\n')if len(x)>1 else None, get_sea()))
    
with open("gback_ensg.txt",'w') as f:
    list(map(lambda x: f.writelines(x[:15]+'\n')if len(x)>1 else None, df_topics.values.ravel()[[str(s)!='nan' for s in df_topics.values.ravel()]]))

In [10]:
gs.get_library_name()

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'Cancer_Cell_Line_Encyclopedia',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'DisGeNET',
 'Disease_Perturbations_from_GEO_down',
 'Disease_Perturbations_from_GEO_up',
 'Disease_Signatures_from_GEO_down_2014',
 'Disease_Signatures_from_GEO_up_2014',
 'DrugMatrix',
 'Drug_Perturbations_from_GEO_2014',
 'Drug_Perturbations_from_GEO_down',
 'Drug_Perturb

In [11]:
#https://www.gsea-msigdb.org/gsea/downloads.jsp
gene_sets = ['GO_Molecular_Function_2018',
             'GO_Biological_Process_2018',
             'GO_Cellular_Component_2018',
             'Human_Phenotype_Ontology',
             'WikiPathways_2019_Human',
             '/home/jovyan/work/phd/MSigDB/c1.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c2.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c3.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c4.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c5.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c6.all.v7.1.symbols.gmt',
             '/home/jovyan/work/phd/MSigDB/c7.all.v7.1.symbols.gmt',
            ]

In [ ]:
threshhold = 5e-1
cutoff = 5e-1
background = len([g for g in get_sea()])
os.system("mkdir -p gsea")
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("gsea/gsea_level_%d_topic_%d.csv"%(l,itopic+1), index_col=[0])
        print(topic)
    except:
        try:
            gene_list = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
            print(topic)
            enriched_topic = get_ontology_df(gene_list, cutoff=cutoff, threshhold = threshhold, gene_sets = gene_sets, background=background)
            enriched_topic = enriched_topic.sort_values(by=['Adjusted P-value'], ascending=True)[:20]
            enriched_topic.to_csv("gsea/gsea_level_%d_topic_%d.csv"%(l,itopic+1))
        except:
            print(*sys.exc_info())
            continue
    print(enriched_topic)

Topic 1
                                                   Term  Adjusted P-value  \
113           neutrophil mediated immunity (GO:0002446)      3.806490e-19   
112   neutrophil activation involved in immune respo...      4.565767e-19   
111               neutrophil degranulation (GO:0043312)      7.287136e-19   
790                 ficolin-1-rich granule (GO:0101002)      7.873850e-12   
791                       tertiary granule (GO:0070820)      6.855181e-09   
792        ficolin-1-rich granule membrane (GO:0101003)      9.533894e-08   
793                secretory granule lumen (GO:0034774)      5.425050e-05   
114                  inflammatory response (GO:0006954)      6.184760e-05   
794              tertiary granule membrane (GO:0070821)      1.466165e-04   
795           ficolin-1-rich granule lumen (GO:1904813)      1.941121e-04   
116              cellular defense response (GO:0006968)      2.853182e-04   
115          defense response to bacterium (GO:0042742)      3.33177

2020-04-03 16:17:54,589 Warning: No enrich terms using library CUSTOM140535831870992 when cutoff = 0.5
2020-04-03 16:17:55,986 Warning: No enrich terms using library CUSTOM140535831871232 when cutoff = 0.5
2020-04-03 16:17:57,319 Warning: No enrich terms using library CUSTOM140535831359648 when cutoff = 0.5
2020-04-03 16:17:57,869 Warning: No enrich terms using library CUSTOM140535831359808 when cutoff = 0.5


                                                   Term  Adjusted P-value  \
70                      muscle contraction (GO:0006936)      2.654274e-21   
72           actin-myosin filament sliding (GO:0033275)      5.166076e-14   
71                 muscle filament sliding (GO:0030049)      7.749114e-14   
467           Striated Muscle Contraction Pathway WP383      2.681280e-12   
1517                                           GNF2_TTN      9.955159e-12   
73                      myofibril assembly (GO:0030239)      4.600890e-11   
74             striated muscle contraction (GO:0006941)      3.379010e-10   
75                       heart contraction (GO:0060047)      2.269503e-09   
347          Muscle fiber inclusion bodies (HP:0100299)      2.010768e-08   
77       actomyosin structure organization (GO:0031032)      5.681283e-08   
76              cardiac muscle contraction (GO:0060048)      5.871863e-08   
1959                         GO_MUSCLE_FILAMENT_SLIDING      2.020521e-07   

2020-04-03 16:18:04,537 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.5
2020-04-03 16:18:06,977 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.5
2020-04-03 16:18:09,750 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.5
2020-04-03 16:18:12,866 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.5
2020-04-03 16:18:20,300 Warning: No enrich terms using library CUSTOM140535573593376 when cutoff = 0.5
2020-04-03 16:18:21,066 Warning: No enrich terms using library CUSTOM140535573592976 when cutoff = 0.5
2020-04-03 16:18:22,766 Warning: No enrich terms using library CUSTOM140535573593056 when cutoff = 0.5
2020-04-03 16:18:22,917 Warning: No enrich terms using library CUSTOM140535571964704 when cutoff = 0.5
2020-04-03 16:18:24,469 Warning: No enrich terms using library CUSTOM140535573590816 when cutoff = 0.5
2020-04-03 16:18:24,576 Warning: No enrich terms using 

                                          Term  Adjusted P-value  \
243  Prader-Willi and Angelman Syndrome WP3998          0.021605   

                    Gene_set  
243  WikiPathways_2019_Human  
Topic 8


...

2020-04-03 16:55:55,512 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.5
2020-04-03 16:55:57,419 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.5
2020-04-03 16:55:59,177 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.5
2020-04-03 16:56:01,648 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.5
2020-04-03 16:56:04,102 Warning: No enrich terms using library WikiPathways_2019_Human when cutoff = 0.5
2020-04-03 16:56:04,233 Warning: No enrich terms using library CUSTOM140535534240304 when cutoff = 0.5
2020-04-03 16:56:04,949 Warning: No enrich terms using library CUSTOM140535534241584 when cutoff = 0.5


                                                  Term  Adjusted P-value  \
45                 BOYAULT_LIVER_CANCER_SUBCLASS_G6_DN          0.220258   
100                   WANG_METASTASIS_OF_BREAST_CANCER          0.220258   
98                             WAGNER_APO2_SENSITIVITY          0.220258   
92       REACTOME_TERMINATION_OF_O_GLYCAN_BIOSYNTHESIS          0.220258   
58   KEGG_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_AND_...          0.220258   
59                            LIU_IL13_MEMORY_MODEL_UP          0.220258   
91   REACTOME_REACTIONS_SPECIFIC_TO_THE_COMPLEX_N_G...          0.220258   
75           REACTOME_BLOOD_GROUP_SYSTEMS_BIOSYNTHESIS          0.220258   
77                            REACTOME_DECTIN_2_FAMILY          0.220258   
78   REACTOME_DEFECTIVE_C1GALT1C1_CAUSES_TN_POLYAGG...          0.220258   
79   REACTOME_DEFECTIVE_GALNT3_CAUSES_FAMILIAL_HYPE...          0.220258   
86   REACTOME_N_GLYCAN_ANTENNAE_ELONGATION_IN_THE_M...          0.220258   
330         

2020-04-03 16:56:13,417 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.5
2020-04-03 16:56:15,119 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.5
2020-04-03 16:56:17,470 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.5
2020-04-03 16:56:19,245 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.5
2020-04-03 16:56:20,955 Warning: No enrich terms using library WikiPathways_2019_Human when cutoff = 0.5


                                                  Term  Adjusted P-value  \
183                                 KRAS.LUNG_UP.V1_DN          0.077672   
136                                         MODULE_154          0.077936   
137                                         MODULE_297          0.077936   
138                                         MODULE_357          0.077936   
65            SCHUETZ_BREAST_CANCER_DUCTAL_INVASIVE_DN          0.133289   
180                      KRAS.600.LUNG.BREAST_UP.V1_DN          0.150811   
259      GSE9946_IMMATURE_VS_LISTERIA_INF_MATURE_DC_DN          0.187079   
208         GSE17721_PAM3CSK4_VS_GADIQUIMOD_2H_BMDC_UP          0.187079   
209             GSE17721_POLYIC_VS_PAM3CSK4_8H_BMDC_UP          0.187079   
210          GSE18791_CTRL_VS_NEWCASTLE_VIRUS_DC_1H_UP          0.187079   
211  GSE19888_CTRL_VS_T_CELL_MEMBRANES_ACT_MAST_CEL...          0.187079   
212  GSE19941_IL10_KO_VS_IL10_KO_AND_NFKBP50_KO_UNS...          0.187079   
213  GSE1994

2020-04-03 16:56:27,316 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.5
2020-04-03 16:56:29,324 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.5
2020-04-03 16:56:30,979 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.5
2020-04-03 16:56:32,678 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.5
2020-04-03 16:56:34,688 Warning: No enrich terms using library WikiPathways_2019_Human when cutoff = 0.5


In [ ]:
topic_pvalues = []
topic_gos = []
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/gsea/gsea_level_%d_topic_%d.csv"%(directory,l,itopic+1))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
            for goc in enriched_topic['Gene_set'][:10].unique():
                topic_gos.append(goc)
        print(topic)
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure()
x = np.arange(1,1+len(topic_pvalues))
c, _, _ = plt.hist(topic_pvalues, histtype='step', lw=2)
plt.plot([-np.log10(0.05) for _ in np.linspace(1,10,num=10)],np.arange(0,np.max(c)+5,(np.max(c)+5)/10), ls='--', lw=5, label="$\\alpha=0.05$")
plt.xlabel('-log(P-value)', fontsize=16)
plt.ylabel("number of topics")
#plt.ylim(0,0.055)
#plt.yscale('log')
plt.legend(fontsize=16)
fig.savefig("%s/pvaluescrosstopic(%d).png"%(directory,l))

In [ ]:
fig = plt.figure(figsize=(20,10))
gos, goscounts = np.unique(topic_gos,return_counts=True)
plt.barh(["\n".join(wrap(str(l).replace('_',' '),20)) for l in gos], goscounts)
plt.yticks(fontsize=15)
plt.show()
fig.savefig("%s/pvaluecategories(%d).pdf"%(directory,l))

# WGCNA vs hSBM

In [ ]:
from scipy.stats import hypergeom
from sklearn.metrics import v_measure_score
import seaborn as sns
sns.set_context('paper')

In [ ]:
hsbm_list = pd.read_csv("topsbm/topsbm_level_2_word-dist.csv", index_col=0).apply(pd.Series.idxmax,axis=1)
hsbm_list.index = [g[:15] for g in hsbm_list.index]
hsbm_list

In [ ]:
wgcna_list = pd.read_csv("wgcna/wgcna_level_0_word-dist.csv", index_col=0).apply(pd.Series.idxmax,axis=1)

In [ ]:
tm_list = pd.read_csv("tm/tm_level_0_word-dist.csv", index_col=0).apply(pd.Series.idxmax,axis=1)

In [ ]:
lda_list = pd.read_csv("lda/lda_level_2_word-dist.csv", index_col=0).apply(pd.Series.idxmax,axis=1)

In [ ]:
list_1 = hsbm_list
list_2 = lda_list

#to uniform
list_1 = list_1[list_1.index.isin(list_2.index)]
list_2 = list_2[list_2.index.isin(list_1.index)]

In [ ]:
population_size = len(list_1[list_1.index.isin(list_2.index)])
pop_successes = {module:len(list_2[list_2==module]) for module in list_2.unique()}
sample_sizes = {topic:len(list_1[list_1==topic]) for topic in list_1.unique()}
num_successes = pd.DataFrame(index=list_1.unique(), columns=list_2.unique()).fillna(0)
for g in list_2.index:
    if g in list_1:
        num_successes.at[list_1[g],list_2[g]]+=1

In [ ]:
df_cmap=pd.DataFrame(index=list_1.unique(), columns=list_2.unique()).fillna(0.5)
for module in df_cmap.columns:
    for topic in df_cmap.index:
        x = num_successes.at[topic,module].astype(int) # number of successes
        M = population_size # pop size
        k = pop_successes[module] # successes in pop
        N = sample_sizes[topic] # sample size
        pval = hypergeom.sf(x-1, M, k, N)
        df_cmap.at[topic,module]=-np.log10(float(pval))

In [ ]:
df_cmap[df_cmap<3]=0

In [ ]:
#df_cmap = df_cmap.sort_values(by=[c for c in df_cmap.columns], axis=0, ascending=True)
#create a color palette with the same number of colors as unique values in the Source column
network_pal = sns.color_palette('husl',n_colors=len(df_cmap.columns))

#Create a dictionary where the key is the category and the values are the
#colors from the palette we just created
network_lut = dict(zip(df_cmap.columns, network_pal))
network_col = df_cmap.columns.map(network_lut)
#Create a dictionary where the key is the category and the values are the
#colors from the palette we just created
network_lut = dict(zip(df_cmap.columns, network_pal))
network_col = df_cmap.columns.map(network_lut)

fig = plt.figure()
cm = sns.clustermap(df_cmap, 
                    row_cluster=False, 
                    col_cluster=False, 
                    metric='euclidean', 
                    vmin=0, 
                    cmap='Blues_r', 
                    col_colors=network_col)

ax = cm.ax_heatmap
ax.tick_params(labelsize=15)
cax = cm.ax_cbar
cax.tick_params(labelsize=24)
cax.set_title("-Log(P-value)", fontsize=30)

cm.fig.suptitle('Algorithm comparison', fontsize=40)
cm.fig.savefig("topics_logp.pdf")

In [ ]:
print("hsbm - lda %.3f"%v_measure_score(list_1, list_2))

In [ ]:
list(map(print,list_1[list_1=="Topic 39"].index))

In [ ]:
print("hsbm - wgcna %.3f"%v_measure_score(hsbm_list, wgcna_list))
print("hsbm - tm %.3f"%v_measure_score(hsbm_list.reindex_like(tm_list), tm_list))
print("tm - wgcna %.3f"%v_measure_score(tm_list, wgcna_list.reindex_like(tm_list)))
print("hsbm - lda %.3f"%v_measure_score(hsbm_list, lda_list.reindex_like(hsbm_list)))